# Fine-tuning Embeddings for RAG

**Dependencies and OpenAI key**

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
!pip install -qU langchain_openai langchain_huggingface langchain_core==0.2.38 langchain langchain_community==0.2.16 langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49

In [3]:
!pip install -qU pymupdf nltk==3.9.1 langchain-qdrant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 21.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.2

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [36]:
from vars import CONTENT_DIR
os.chdir(CONTENT_DIR)

**Load and Chunk the Data**



In [8]:
import importlib
import vanilla_rag
from vars import PDFS, FT_CHUNK_SIZE, FT_OVERLAP

importlib.reload(vanilla_rag)
for pdf in PDFS:
    training_documents = await vanilla_rag.load_and_chunk_pdf(pdf,FT_CHUNK_SIZE,FT_OVERLAP)


Loading https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf...
Chunking...
Loading https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf...
Chunking...


In [9]:
print(len(training_documents))
print(training_documents[0])

387
page_content='NIST Trustworthy and Responsible AI  
NIST AI 600-1 
Artificial Intelligence Risk Management 
Framework: Generative Artificial 
Intelligence Profile 
 
 
 
This publication is available free of charge from: 
https://doi.org/10.6028/NIST.AI.600-1'


Next, we're going to associate each of our chunks with a unique identifier.

In [16]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [17]:
training_split_documents = training_documents[:300]
val_split_documents = training_documents[300:340]
test_split_documents = training_documents[340:]

**Create fine tuning dataset**

In [18]:
from langchain_openai import ChatOpenAI
from vars import BASELINE_CHAT_MODEL

qa_chat_model = ChatOpenAI(
    model=BASELINE_CHAT_MODEL,
    temperature=0
)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [19]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [20]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [21]:
import uuid

def process_document(document, n_questions):
    """Generate N questions for a single document"""
    questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})

    doc_questions = {}
    doc_relevant_docs = {}
    for question in questions_generated.content.split("\n"):
        question_id = str(uuid.uuid4())
        doc_questions[question_id] = "".join(question.split(".")[1:]).strip()
        doc_relevant_docs[question_id] = [document.metadata["id"]]

    return doc_questions, doc_relevant_docs

def create_questions(documents, n_questions):
    """Loop over documents and generate questions. This was initially async, but my
    OpenAI rate limits are low, so changed so sync execution."""
    questions = {}
    relevant_docs = {}

    for i, doc in enumerate(documents):
        if i%10 == 0:
            print(f"Document {i}...",end="")
        doc_questions, doc_relevant_docs = process_document(doc, n_questions)
        questions.update(doc_questions)
        relevant_docs.update(doc_relevant_docs)
    return questions, relevant_docs

We'll use the function to generate training, validation, and test data with `n_questions=2` for each.

In [22]:
training_questions, training_relevant_contexts = create_questions(training_split_documents,2)

Document 0...Document 10...Document 20...Document 30...Document 40...Document 50...Document 60...Document 70...Document 80...Document 90...Document 100...Document 110...Document 120...Document 130...Document 140...Document 150...Document 160...Document 170...Document 180...Document 190...Document 200...Document 210...Document 220...Document 230...Document 240...Document 250...Document 260...Document 270...Document 280...Document 290...

In [23]:
val_questions, val_relevant_contexts = create_questions(val_split_documents,2)

Document 0...Document 10...Document 20...Document 30...

In [24]:
test_questions, test_relevant_contexts = create_questions(test_split_documents,2)

Document 0...Document 10...Document 20...Document 30...Document 40...

In [27]:
import json
from vars import FT_TRAIN_DATASET_FILE

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open(FT_TRAIN_DATASET_FILE, "w") as f:
  json.dump(train_dataset, f)

In [34]:
from vars import FT_VAL_DATASET_FILE
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open(FT_VAL_DATASET_FILE, "w") as f:
  json.dump(val_dataset, f)

In [35]:
from vars import FT_TEST_DATASET_FILE
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open(FT_TEST_DATASET_FILE, "w") as f:
  json.dump(test_dataset, f)

In [7]:
import json
import os
from vars import CONTENT_DIR, FT_TRAIN_DATASET_FILE, FT_VAL_DATASET_FILE, FT_TEST_DATASET_FILE

os.chdir(CONTENT_DIR)

with open(FT_TRAIN_DATASET_FILE, 'r') as file:
    train_dataset = [json.loads(line) for line in file]
train_dataset = train_dataset[0]

with open(FT_VAL_DATASET_FILE, 'r') as file:
    val_dataset = [json.loads(line) for line in file]
val_dataset = val_dataset[0]

with open(FT_TEST_DATASET_FILE, 'r') as file:
    test_dataset = [json.loads(line) for line in file]
test_dataset = test_dataset[0]


**Next step: Fine tuning**

In [8]:
!pip install -qU sentence_transformers datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [9]:
from sentence_transformers import SentenceTransformer

from google.colab import userdata
userdata.get('HF_TOKEN')

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/84.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [11]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample
from vars import BATCH_SIZE

# Munge data to expected training format
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

# Create data loader
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [12]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

# Prepare loss function
matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [13]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

# Configure evaluator with validation set
corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [14]:
# Model training
from vars import EPOCHS, FT_MODEL_NAME
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path=FT_MODEL_NAME,
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
30,No log,No log,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.945587,0.927083,0.927083,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.945587,0.927083,0.927083
50,No log,No log,0.887500,0.975000,1.000000,1.000000,0.887500,0.325000,0.200000,0.100000,0.887500,0.975000,1.000000,1.000000,0.948016,0.930625,0.930625,0.887500,0.975000,1.000000,1.000000,0.887500,0.325000,0.200000,0.100000,0.887500,0.975000,1.000000,1.000000,0.948016,0.930625,0.930625
60,No log,No log,0.862500,0.975000,1.000000,1.000000,0.862500,0.325000,0.200000,0.100000,0.862500,0.975000,1.000000,1.000000,0.939337,0.918750,0.918750,0.862500,0.975000,1.000000,1.000000,0.862500,0.325000,0.200000,0.100000,0.862500,0.975000,1.000000,1.000000,0.939337,0.918750,0.918750
90,No log,No log,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.943402,0.924375,0.924375,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.943402,0.924375,0.924375
100,No log,No log,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.943402,0.924375,0.924375,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.943402,0.924375,0.924375
120,No log,No log,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.943402,0.924375,0.924375,0.875000,0.975000,1.000000,1.000000,0.875000,0.325000,0.200000,0.100000,0.875000,0.975000,1.000000,1.000000,0.943402,0.924375,0.924375
150,No log,No log,0.850000,0.975000,1.000000,1.000000,0.850000,0.325000,0.200000,0.100000,0.850000,0.975000,1.000000,1.000000,0.934175,0.911875,0.911875,0.850000,0.975000,1.000000,1.000000,0.850000,0.325000,0.200000,0.100000,0.850000,0.975000,1.000000,1.000000,0.934175,0.911875,0.911875


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

**Push model to Huggingface Hub**

In [16]:
!pip install huggingface_hub


In [22]:
from huggingface_hub import notebook_login
from huggingface_hub import HfApi
from vars import HF_USERNAME, FT_MODEL_NAME

notebook_login()

model.save_pretrained(FT_MODEL_NAME) # save locally

api = HfApi()

repo_name = f"{HF_USERNAME}/{FT_MODEL_NAME}"

try:
    repo_info = api.repo_info(repo_id=repo_name)
except RepositoryNotFoundError:
    api.create_repo(repo_id=repo_name, private=False)

api.upload_folder(
    folder_path=FT_MODEL_NAME,
    repo_id=repo_name,
    repo_type="model"
)



model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/achapman/finetuned_arctic_ai_risk/commit/e77855f217711f7688a52d7f18abd8489f35859a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='e77855f217711f7688a52d7f18abd8489f35859a', pr_url=None, pr_revision=None, pr_num=None)